# INSIDE RSA

(i) Computing the gcd; (ii) Computing the modular multiplicative inverse (when it exists); (iii) Encryption/Decryption with Textbook RSA; (iv) Signature with Full-Domain Hash

In [1]:
from functools import reduce
import random
from Crypto.Util import number
from hashlib import sha256
from base64 import b64encode, b64decode


(i) Computing the gcd

In [2]:

#Implement the Greatest Common Divisor (gcd) for any list of numbers

''' 
numbers: A list of numbers given as an input

gcd function:
In case we have 2 numbers as an input, we return
the (greatest) number until the division between him 
and the other has zero remainder.
In case of more than 2 numbers are given, we use the
reduce function which applies the GCD() for each number
that we have in the list.

For instance if we have numbers = [a,b,c,d]
what reduce function mainly does in our case is:

gcd(a, b, c, d) = gcd(a,b) 
             = gcd(gcd(a, b), c) 
             = gcd(gcd(gcd(a, b), c) , d)

'''
numbers = list(map(int,input().split()))

def gcd(numbers):
    if len(numbers) == 2:
        if numbers[1] == 0:
            return numbers[0]
        else:
            numbers = [numbers[1], numbers[0] % numbers[1]]
            return  gcd(numbers)
    else:
         print("The gcd of",numbers,"is",reduce(lambda x,y: gcd([x,y]), numbers))

        
        
gcd(numbers)

2 2


2

(ii) Computing the modular multiplicative inverse (when it exists)

In [3]:
# Computing the modular multiplicative inverse (when it exists)

'''
x : a positive integer (the divisor)
p : a positive integer (the dividend)
Given (x,p) we compute the inverse of x if it exists.
Here the parameter numbers = [x,p] 

'''
print("Specify the numbers") 
x,p= list(map(int,input().split()))
print("The divisor is",x)
print("The dividend is",p)


def inverse(x,p):
    if x==1 or p==1:
        return 1
    if gcd([x,p])!=1:
         print("The  modular multiplicative inverse of" ,x,"with",p,"as a dividend does not exist")     
    else:
        for i in range(1,p):
            if ((x * i) % p == 1):
                print("The modular multiplicative inverse of", x ,"is",i) 
                

inverse(x,p)


Specify the numbers
2 2
The divisor is 2
The dividend is 2
The  modular multiplicative inverse of 2 with 2 as a dividend does not exist


 (iii) Encryption/Decryption with Textbook RSA

INPUT: Security parameter l
OUTPUT: RSA public key e, private key d and n
1. Randomly select two primes p and q with same bitlength l/2
2. Compute n = pq and phi = (p-1)(q-1)
3. Select an arbitrary integer e with 1 < e < phi and gcd(e, phi)==1
4. Compute the integer d satisfying 1 < d < phi and ed == 1 mod phi
5. Return(n, e, d)


Algorithm 1.2: Basic RSA encryption
INPUT: RSA public key e, n, plaintext m
OUTPUT: Ciphertext c
1. Compute c = m**e mod n
2. Return(c)
Algorithm 1.3: Basic RSA decryption
INPUT: RSA private d, n, ciphertext c
OUTPUT: Plaintext m
1. Compute m = c**d mod n
2. Return(m)

In [4]:
#Generate random prime numbers with specific length of bits

def prime_gen(bit_length):
    #random.seed(123)
    bits = bit_length//2
    prime = number.getPrime(bits)
    return (prime)

In [5]:
#We choose the encryption "e" such that gcd(e,φ(N))=1
'''
Input : the value of "φ" function
Output : the encryption "e"
We randomly pick a candidate number less than φ(Ν)
and check if the gcd(e,φ(N))=1
'''
def encrypt_pow(phi_N):
    candidate = [i for i in range(1, phi_N)]
    e = random.choice(candidate)
    while(gcd([e, phi_N]) != 1):
        e = random.choice(candidate)
    return e
#encrypt_pow(8)        

In [6]:
## Same function used in RSA textbook

def inverse_rsa(x,p):
    if x==1 or p==1:
        return 1
    if gcd([x,p])!=1:
         print("The  modular multiplicative inverse does not exist")     
    else:
        for i in range(1,p):
            if ((x * i) % p == 1):
                return i
                

inverse_rsa(7,9)

4

In [7]:
# Generate the public key adn the private key

'''
Input : number of bits which will define the prime bitlength
Output: The primes (p,q),their product (N) and the function phi_N

'''
def Key_Generation(bits):
    # Randomly select 2 primes with same Bitlength l/2
    p = prime_gen(bits)
    q = prime_gen(bits)
    # Compute
    N = p * q
    phi_N = (p - 1) * (q - 1)
    # Select an arbitrary integer e with 1 < e < phi and gcd(e,phi) == 1
    e = encrypt_pow(phi_N)
    # Compute the integer d statisfying 1 < d < phi and e*d == 1 mod phi
    d = inverse_rsa(e, phi_N)
    # Return n e d
    print("Public Key: " ,e)
    print("Private Key: " ,d)
    print("N =",N)
    return(p,q,N,phi_N,e,d)

In [32]:
Key_Generation(16)

Public Key:  17543
Private Key:  15703
N = 33823


(227, 149, 33823, 33448, 17543, 15703)

RSA Encryption Scheme

Input: The plaintext m,the RSA public key e and n
    
Output: The ciphertext c
    
1. Compute c = m^e mod n
2. Return(c)

In [9]:
#Encryption function

'''
Input : m (the plaintext) ,e (encr code-public key) , n (the prime product)
Output : c (the ciphertext)
#ord? - Return the Unicode code point for a one-character string -in our case the encoding of each letter
#pow? - x**y % z (with three arguments) here x is the letter enc, y is e and z is n

'''

def encryption(m,e,N):
    c = []
    for x in m:
        c.append(pow(ord(x),e,N))
    #print(int(''.join(map(lambda x: str(x), c))))    
    return c



RSA Decryption Scheme

Input:The ciphertext c,the RSA private key d, and  N
    
Output: The plaintext m
    
1. Compute m = c**d mod n
2. Return(m)

In [10]:
#Decryption function

'''
Input : c (the ciphertext),d (decr code-private key) , N (the prime product)
Output : m (the plaintext) 
#chr? - Return a Unicode string of one character with ordinal i; 
(it's like decrypting its unicode code as a string bit)
#pow? - x**y % z (with three arguments) here x is the letter enc, y is e and z is n

'''


def decryption(c,d,N):
    m = []
    for x in c:
        m.append(chr(pow(x,d,N)))
    return ''.join(m)




Decrypt and get the original message (plaintext)

In [71]:
m = "CRYPTO IS COOL"
p,q,N,phi_N,e,d = Key_Generation(16)
c = encryption(m,e,N) 
d = decryption(c,d,N)
print("The decrypted ciphertext c is:",d )


Public Key:  2857
Private Key:  28393
N = 31439
The decrypted ciphertext c is: CRYPTO IS COOL


(iv) Signature with Full-Domain Hash

In [15]:
#Hash the messagewith SHA-256

'''
Input : message (the plaintext)
Output : The hashed message
Procedure : To generate a signature, make a hash from the plaintext, 
encrypt it with your private key, include it alongside the plaintext.
'''



def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed


In [16]:
#Authentication Procedure

'''
Input : receivedeHash (the hash we received after decryption),message (the plaintext)
Output : The result of verification 

Procedure : Make a hash from the plaintext (ourHashed), decrypt the signature 
with the sender's public key (decrypted_msg),
check that both hashes are the same (receivedHash ~ ourHashed)
'''



def verify(receivedHash, message):
    ourHash = hashFunction(message)
    if receivedHash == ourHash:
        print(1)
        print("Authentication successful: ", )
        print(receivedHash, " = ", ourHash)
    else:
        print(0)
        print("Authentication failed")
        print(receivedHash, " != ", ourHash)
    


Main function

Input : Number of bits which define the bitlength (bits), the plaintext (message)
Output : The public-private generated key pair, the encrypted-decrypted message, the results of the verification process



In [42]:
def main():
    bits = int(input("Specify the number of bits for the prime generation:"))
    message = str(input("Enter a message to encrypt with your private key: "))
    #Prime Generation,private and public key
    p = prime_gen(bits)
    q = prime_gen(bits)
    N = p * q
    phi_N = (p - 1) * (q - 1)
    e = encrypt_pow(phi_N)
    d = inverse_rsa(e, phi_N)
    print("")
    print("Public Key: " ,e)
    print("Private Key: " ,d)
    print("N =",N)
    #Encryption and Decryption
    c = encryption(message,e,N)
    print("")
    print("The encrypted message with public key e =",e,"is c=")
    print(int(''.join(map(lambda x: str(x), c))))
    print("")
    print("The dectrypted message with private key d =",d,"is")
    print(decryption(c,d,N))  
    #Sign the message and verify
    hashed = hashFunction(message)
    print("")
    print("Encrypting message with private key d = ", d ," . . .")
    encrypted_msg = encryption(hashed,d,N)   
    print("Your signature is : tag =")
    print(''.join(map(lambda x: str(x), encrypted_msg)))
    print("")
    print("Decrypting message with public key e = ", e ," . . .")
    decrypted_msg = decryption(encrypted_msg,e,N)
    print("Your decrypted message is: tag =")  
    print(decrypted_msg)
    print("")
    print("Verification results~")
    message = str(input("Enter a message to encrypt with your private key: "))
    verify(decrypted_msg, message)


main()    

Specify the number of bits for the prime generation:16
Enter a message to encrypt with your private key: CRYPTO IS COOL

Public Key:  2857
Private Key:  28393
N = 31439

The encrypted message with public key e = 2857 is c=
234882270229371120863565254569955559164776995234882545254527370

The dectrypted message with private key d = 28393 is
CRYPTO IS COOL

Encrypting message with private key d =  28393  . . .
Your signature is : tag =
5955251718777283188777311679212211502517114951178022489314951149511158023529595587772489314951595514951877787771973031167248932831823529921223475595511580311671973087779212304741780223475283188777251719212921214951283181158017802921223529115803047421150311672347559555955304743116714951149512352924893

Decrypting message with public key e =  2857  . . .
Your decrypted message is: tag =
538c8d12394b99f058b959887dbc0165fd781a46c83119cf410fa2d655ad990b

Verification results~
Enter a message to encrypt with your private key: CRYPTO IS COOL
1
Authentication succe